# Search Tools

LangChain provides the framework for agents and tools

The langchain-google-genai connector lets us use Gemini models, and the google-generativeai SDK enables communication with Gemini itself.

From the wifipedia package, we pull in the interface for Wikipedia queries. From LangChain, we import the tool decorator the Gemini Chat model wrapper PromptTemplates and the agent utilities to tie everything together.

Finally, we set the wikipedia language to Englsih, so that all results come back in a consistent format.

_format_hits takes the raw search results from DuckDuckGo and formats them into a clean list. It loops through the first few results, pulls out the title, url and snippet, then arranges them in a numbered format.
If no results are found, it simply returns No results.
raw API responses arent always learner-friendly
We often need a formatter to present them clearly to the agent or to the user.

web_search - runs a DuckDuckGo search for any query returning a compact list of results with titles, URLs and snippets.
We can also set how many results to return and even filter by recency,like the past day, week, month or year. In the function we call the DuckDuckGo client, collect the hits, and format them neatly using our helper.
If something goes wrong, the tool returns a clear error message instead of crashing.
This is how you wrap a search API, into a reusable LangChain tool, giving our agent the ability to look up live information on demand.

wiki_summary it takes a topic name and returns a short Wikipedia summary along with the page URL. Inside, it first loads the page, then extracts a few sentences from the summary. If the topic is ambiguous, it warns the user and shows some options.
If the page doesnt exist, it returns a clear message, and if any other error happens, it also handled gracefully.
Wraps Wikipedia into a reliable knowledge source for our agent, so it can pull quick trustworthy summaries when users ask about a topic.

In [1]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 9.9 MB/s eta 0:00:00


In [2]:
!pip install -qU ddgs # duck duck go library;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.4 MB/s eta 0:00:00


In [3]:
!pip install -qU wikipedia

  Preparing metadata (setup.py) ... done


In [5]:
!pip install -qU langchain-classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_GEMINI_API")  # paste your key
print(" API key set.")

 API key set.


# Define tools (DuckDuckGo + Wikipedia)

In [6]:
from typing import List
from ddgs import DDGS
import wikipedia

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI       # to work with Gemini models
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor, create_react_agent

wikipedia.set_lang("en")

def _format_hits(hits: List[dict], k: int = 5) -> str:
    lines = []
    for i, h in enumerate(hits[:k], 1):
        title = h.get("title") or h.get("heading") or ""
        url = h.get("href") or h.get("url") or ""
        snippet = h.get("body") or h.get("description") or h.get("snippet") or ""
        lines.append(f"{i}. {title}\n   {url}\n   {snippet}")
    return "\n".join(lines) if lines else "No results."

@tool
def web_search(query: str, max_results: int = 5, time_range: str = None) -> str:
    """DuckDuckGo web search. Returns up to `max_results` compact results with URLs and snippets.
    time_range can be 'd','w','m','y' (day, week, month, year) for recency, or None."""
    try:
        with DDGS() as ddg:
            hits = list(ddg.text(query, max_results=max_results, timelimit=time_range))
        return _format_hits(hits, k=max_results)
    except Exception as e:
        return f"Search error: {e}"

@tool
def wiki_summary(topic: str, sentences: int = 3) -> str:
    """Return a short Wikipedia summary with the canonical page URL."""
    try:
        page = wikipedia.page(topic, auto_suggest=True, redirect=True)
        summary = wikipedia.summary(page.title, sentences=sentences)
        return f"Title: {page.title}\nURL: {page.url}\nSummary: {summary}"
    except wikipedia.DisambiguationError as e:
        opts = "; ".join(e.options[:5])
        return f"Disambiguation: be specific. Options include: {opts}"
    except wikipedia.PageError:
        return "No matching page found."
    except Exception as e:
        return f"Wikipedia error: {e}"

# Build the search agent

- build_search_agent() - Then, we register the two tools we created web_search for fresh or news like queries and wiki_summary for background and definitions.

Next,we define a custom system prompt
it tells the agent to always use tools for facts, numbers or dates and explains when to choose search vs Wikipedia.
It also reminds the agent to cite sources with titles and URLs keep answers concise, and never hallucinate.

create a tool_calling_agent with these rules and wrap it in Executor which gives us a simple .invoke interface

At the bottom, we call the function to initialize the agent and print a message confirming its ready.
Combining multiple tools under one agent and guiding it with a well designed prompt, we can create a reliable, research assistant that choose the right source for the right job.

In [7]:
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
def build_search_agent():
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)
    tools = [web_search, wiki_summary]

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a research assistant.\n"
         "Use tools for ANY factual claims, numbers, dates, or 'latest' info.\n"
         "Choose:\n"
         "- web_search: for fresh/news or when URLs are required.\n"
         "- wiki_summary: for background/definitions.\n"
         "When you cite, include page titles and URLs from tool outputs.\n"
         "Keep answers concise. Prefer top 3 results. Do not hallucinate."
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)  # clean output

search_agent = build_search_agent()
print(" Search agent ready.")

 Search agent ready.


Helper function called ask that takes in a question, sends it to the agent, and prints the output

# Quick tests

In [9]:
def ask(q: str):
    res = search_agent.invoke({"input": q})
    print(res["output"], "\n")

# 1) Simple fact + citation
ask("Who won the FIFA World Cup 2022? Give one sentence and include a source URL.")

[{'type': 'text', 'text': 'Argentina won the FIFA World Cup 2022, defeating France in the final. Source: https://en.wikipedia.org/wiki/2022_', 'extras': {'signature': 'CiQBvj72+8ZeajAOv+y9DiVH/XchxbisAtqyo9yYYGcYhVMSRMsKbQG+Pvb7DJcch/ZwGkpevAGYmhkMfrL5Htu55OVWBzrFDIW4DpscQ8bFhmCE+u0SAOxJg4JOOVdk9j4CiO/a9iuczJLfp+nFXqvDCVKcVUlwIJbGvzfx43nME+x0Ed/vREPE+DL9rT2qeUVL9m8KWAG+Pvb70UiHp/AXTZjrfPU9sqt6etWLwOPunYjCcLPXPZaUZ2x9Q7sTs7WUIXscR1pXYweclClQIAZ6fs/jXByU4BBojDxSns9QOZkwt/VIgCzjymvhYNo='}, 'index': 0}, 'FIFA_World_Cup'] 



In [11]:
# 3) Fresh headlines (limit and summarize)
ask("Find 3 recent headlines about generative AI this week. One line per item with URL.")

Here are 3 recent headlines about generative AI:

1.  Free AI Headline Generator | Create Catchy Headlines Instantly
    https://www.grammarly.com/ai/ai-writing-tools/headline-generator
2.  Generative AI Headlines - GINGER LIU
    https://www.gingerliu.com/generative-ai-headlines-3/
3.  Listen to Generative AI Headlines 4–4–2
    https://www.linkedin.com/pulse/listen-generative-ai-headlines-442-ginger-liu-m-f-a--ero9e 

